# RAG (Retrieval-Augmented Generation) Assignment

## Problem Statement
Build a Question-Answering system using RAG architecture that can answer questions based on PDF documents. The system will:
1. Load and process PDF documents
2. Create vector embeddings for efficient retrieval
3. Use Google's Gemini model for generating accurate responses based on retrieved context

---

## Dataset / Knowledge Source
- **Type of Data**: PDF documents
- **Data Source**: Sample PDF documents (can be research papers, manuals, or any text-based PDFs)
- **Location**: `data/` folder

---

## RAG Architecture

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                           RAG PIPELINE ARCHITECTURE                          │
└─────────────────────────────────────────────────────────────────────────────┘

                              INDEXING PHASE
┌──────────┐    ┌──────────────┐    ┌─────────────┐    ┌────────────────┐
│  PDF     │───▶│   Text       │───▶│   Text      │───▶│   Embedding    │
│  Docs    │    │   Extraction │    │   Chunking  │    │   Generation   │
└──────────┘    └──────────────┘    └─────────────┘    └────────────────┘
                                                               │
                                                               ▼
                                                       ┌────────────────┐
                                                       │  FAISS Vector  │
                                                       │    Store       │
                                                       └────────────────┘
                                                               │
                              RETRIEVAL PHASE                  │
┌──────────┐    ┌──────────────┐    ┌─────────────┐           │
│  User    │───▶│   Query      │───▶│  Similarity │◀──────────┘
│  Query   │    │   Embedding  │    │   Search    │
└──────────┘    └──────────────┘    └─────────────┘
                                           │
                                           ▼
                              GENERATION PHASE
                       ┌─────────────────────────┐
                       │   Top-K Relevant Chunks │
                       └─────────────────────────┘
                                   │
                                   ▼
┌──────────────────────────────────────────────────────────────┐
│                    PROMPT TEMPLATE                           │
│  Context: {retrieved_chunks}                                 │
│  Question: {user_query}                                      │
│  Answer based on context only.                               │
└──────────────────────────────────────────────────────────────┘
                                   │
                                   ▼
                       ┌─────────────────────────┐
                       │   Google Gemini LLM     │
                       └─────────────────────────┘
                                   │
                                   ▼
                       ┌─────────────────────────┐
                       │   Generated Response    │
                       └─────────────────────────┘
```

## Step 1: Install and Import Dependencies

In [ ]:
# Install required packages (uncomment if needed)
# !pip install -r requirements.txt

In [ ]:
# Import required libraries
import os
from dotenv import load_dotenv

# LangChain components
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()

# Verify API key is loaded
if not os.getenv("GOOGLE_API_KEY"):
    raise ValueError("GOOGLE_API_KEY not found. Please set it in .env file")

print("✅ All dependencies loaded successfully!")

## Step 2: Load PDF Documents

We use `PyPDFLoader` to extract text from PDF files. The `DirectoryLoader` allows loading multiple PDFs from a folder.

In [ ]:
# Define the data directory
DATA_PATH = "data/"

def load_documents(data_path):
    """
    Load all PDF documents from the specified directory.
    Returns a list of Document objects with page content and metadata.
    """
    loader = DirectoryLoader(
        data_path,
        glob="**/*.pdf",
        loader_cls=PyPDFLoader,
        show_progress=True
    )
    documents = loader.load()
    return documents

# Load documents
documents = load_documents(DATA_PATH)
print(f"✅ Loaded {len(documents)} pages from PDF documents")

# Display sample content
if documents:
    print(f"\n📄 Sample content from first page:")
    print(f"Source: {documents[0].metadata.get('source', 'Unknown')}")
    print(f"Content preview: {documents[0].page_content[:500]}...")

## Step 3: Text Chunking Strategy

### Chunking Configuration
- **Chunk Size**: 1000 characters
- **Chunk Overlap**: 200 characters

### Reason for Chosen Strategy
1. **RecursiveCharacterTextSplitter**: Splits text hierarchically (paragraphs → sentences → words) preserving semantic coherence
2. **Chunk Size (1000)**: Optimal balance between:
   - Enough context for meaningful retrieval
   - Not too large to dilute relevance
   - Fits within embedding model context limits
3. **Overlap (200)**: Ensures continuity between chunks, preventing loss of context at boundaries

In [ ]:
# Text chunking configuration
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

def chunk_documents(documents, chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP):
    """
    Split documents into smaller chunks for embedding.
    Uses RecursiveCharacterTextSplitter for semantic preservation.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]  # Hierarchical splitting
    )
    chunks = text_splitter.split_documents(documents)
    return chunks

# Create chunks
chunks = chunk_documents(documents)
print(f"✅ Created {len(chunks)} chunks from {len(documents)} pages")
print(f"\n📊 Chunking Statistics:")
print(f"   - Chunk Size: {CHUNK_SIZE} characters")
print(f"   - Chunk Overlap: {CHUNK_OVERLAP} characters")
print(f"   - Average chunks per page: {len(chunks)/max(len(documents),1):.1f}")

## Step 4: Create Embeddings

### Embedding Model Details
- **Model**: Google's `models/embedding-001`
- **Dimension**: 768

### Reason for Selection
1. **High Quality**: Google's embedding model provides state-of-the-art semantic understanding
2. **Integration**: Seamless integration with Google Gemini for generation
3. **Cost-Effective**: Free tier available for development
4. **Multilingual**: Supports multiple languages out of the box

In [ ]:
# Initialize embedding model
def get_embeddings():
    """
    Initialize Google Generative AI Embeddings.
    Uses the embedding-001 model for text embeddings.
    """
    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001"
    )
    return embeddings

embeddings = get_embeddings()
print("✅ Embedding model initialized: models/embedding-001")

# Test embedding generation
test_embedding = embeddings.embed_query("test query")
print(f"   - Embedding dimension: {len(test_embedding)}")

## Step 5: Create FAISS Vector Store

### Vector Database Details
- **Database**: FAISS (Facebook AI Similarity Search)

### Reason for Selection
1. **Speed**: Optimized for fast similarity search on CPU
2. **Memory Efficient**: Uses efficient indexing structures
3. **Local Storage**: No external database required
4. **Scalability**: Handles millions of vectors efficiently

In [ ]:
# Vector store configuration
VECTOR_STORE_PATH = "vectorstore/faiss_index"

def create_vector_store(chunks, embeddings, store_path=VECTOR_STORE_PATH):
    """
    Create FAISS vector store from document chunks.
    Saves the index locally for reuse.
    """
    # Create vector store
    vector_store = FAISS.from_documents(
        documents=chunks,
        embedding=embeddings
    )
    
    # Save locally
    vector_store.save_local(store_path)
    print(f"✅ Vector store saved to: {store_path}")
    
    return vector_store

def load_vector_store(embeddings, store_path=VECTOR_STORE_PATH):
    """
    Load existing FAISS vector store from disk.
    """
    vector_store = FAISS.load_local(
        store_path,
        embeddings,
        allow_dangerous_deserialization=True
    )
    return vector_store

# Create or load vector store
if chunks:
    vector_store = create_vector_store(chunks, embeddings)
    print(f"✅ Vector store created with {len(chunks)} vectors")
else:
    print("⚠️ No chunks to index. Please add PDF files to the data/ folder.")

## Step 6: Setup Retrieval Chain

Configure the retriever and LLM for question answering.

In [ ]:
# Configure retriever
TOP_K = 4  # Number of relevant chunks to retrieve

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": TOP_K}
)

print(f"✅ Retriever configured with top-{TOP_K} similarity search")

In [ ]:
# Initialize LLM (Google Gemini)
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.3,  # Lower temperature for factual responses
    max_tokens=1024
)

print("✅ LLM initialized: gemini-1.5-flash")

In [ ]:
# Define prompt template
PROMPT_TEMPLATE = """
You are a helpful assistant that answers questions based on the provided context.
Use ONLY the information from the context to answer the question.
If the answer is not found in the context, say "I cannot find this information in the provided documents."

Context:
{context}

Question: {question}

Answer: Provide a clear, concise answer based on the context above.
"""

prompt = PromptTemplate(
    template=PROMPT_TEMPLATE,
    input_variables=["context", "question"]
)

print("✅ Prompt template configured")

In [ ]:
# Create RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Stuff all retrieved docs into context
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

print("✅ RAG pipeline ready!")

## Step 7: Query Function

Helper function to query the RAG system and display results.

In [ ]:
def query_rag(question: str, show_sources: bool = True):
    """
    Query the RAG system with a question.
    Returns the answer and optionally shows source documents.
    """
    print(f"\n{'='*60}")
    print(f"📝 Question: {question}")
    print(f"{'='*60}")
    
    # Get response
    response = qa_chain.invoke({"query": question})
    
    print(f"\n💡 Answer:")
    print(response["result"])
    
    if show_sources and "source_documents" in response:
        print(f"\n📚 Sources ({len(response['source_documents'])} documents):")
        for i, doc in enumerate(response["source_documents"], 1):
            source = doc.metadata.get("source", "Unknown")
            page = doc.metadata.get("page", "N/A")
            print(f"   {i}. {source} (Page: {page})")
    
    print(f"\n{'='*60}\n")
    return response

## Step 8: Test Queries

Testing the RAG system with sample queries.

In [ ]:
# Test Query 1
response1 = query_rag("What is the main topic of the document?")

In [ ]:
# Test Query 2
response2 = query_rag("Can you summarize the key points discussed?")

In [ ]:
# Test Query 3
response3 = query_rag("What are the conclusions or recommendations mentioned?")

## Future Improvements

### 1. Better Chunking Strategies
- **Semantic Chunking**: Use sentence embeddings to create semantically coherent chunks
- **Document Structure Aware**: Respect headers, sections, and paragraphs
- **Sliding Window with Overlap**: Variable overlap based on content density

### 2. Reranking / Hybrid Search
- **Cross-Encoder Reranking**: Use a cross-encoder model to rerank retrieved chunks
- **Hybrid Search**: Combine dense (embedding) and sparse (BM25) retrieval
- **Maximal Marginal Relevance (MMR)**: Diversify retrieved results

### 3. Metadata Filtering
- **Filter by Document**: Allow querying specific documents
- **Filter by Date**: Filter based on document date/version
- **Filter by Section**: Target specific sections of documents

### 4. UI Integration
- **Streamlit Dashboard**: Interactive web interface (see `app.py`)
- **Chat History**: Maintain conversation context
- **Document Upload**: Allow users to upload their own PDFs

### 5. Additional Enhancements
- **Query Expansion**: Rephrase queries for better retrieval
- **Multi-modal RAG**: Support images, tables from PDFs
- **Caching**: Cache embeddings and responses for efficiency
- **Evaluation Metrics**: Implement RAGAS for quality assessment

## Summary

| Component | Choice | Reason |
|-----------|--------|--------|
| Document Loader | PyPDFLoader | Reliable PDF text extraction |
| Text Splitter | RecursiveCharacterTextSplitter | Semantic coherence preservation |
| Chunk Size | 1000 chars | Balance between context and precision |
| Chunk Overlap | 200 chars | Continuity between chunks |
| Embedding Model | Google embedding-001 | High quality, free tier available |
| Vector Store | FAISS | Fast, local, memory efficient |
| LLM | Gemini 1.5 Flash | Cost-effective, fast responses |
| Chain Type | Stuff | Simple, effective for small context |